In [1]:
# Setup library
## install -r requirements.txt
from __future__ import absolute_import, division, print_function, unicode_literals
import os

import matplotlib.pylab as plt
%matplotlib widget

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [2]:
# Classifier from TF hub
import numpy as np

classifier_url = 'https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/2' #@param {type:"string"}
IMAGE_SHAPE = (224, 224)
classifier = tf.keras.Sequential([
    hub.KerasLayer(classifier_url, input_shape=IMAGE_SHAPE+(3, )) # Channel 3 RGB
])

## And labels
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt', 
                                      'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())

In [3]:
# prepare dataset
dataset_root = os.path.abspath(os.path.expanduser('./data/'))
print(f'Dataset root: {dataset_root}')

image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255,
                                                                  validation_split=0.1)
train_data = image_generator.flow_from_directory(dataset_root, target_size=IMAGE_SHAPE,
                                                 subset='training')
validation_data = image_generator.flow_from_directory(dataset_root, target_size=IMAGE_SHAPE,
                                                 subset='validation')

for image_batch, label_batch in validation_data:
    print(f'Image batch shape: {image_batch.shape}')
    print(f'Label batch shape: {label_batch.shape}')
    break

Dataset root: /home/harny/Github/cell-classification/data
Found 842 images belonging to 3 classes.
Found 92 images belonging to 3 classes.
Image batch shape: (32, 224, 224, 3)
Label batch shape: (32, 3)


In [4]:
# Predict batch input example
## using original ImangeNet classifier

import PIL.Image as Image

result_batch = classifier.predict(image_batch)
print(f'Batch result shape: {result_batch.shape}')

predicted_class_names = imagenet_labels[np.argmax(result_batch, axis=-1)]
print(f'Batch predicted class names: {predicted_class_names}')

fig1 = plt.figure(figsize=(10, 9))
fig1.subplots_adjust(hspace=0.5)
for n in range(30):
    ax = fig1.add_subplot(6, 5, n+1)
    ax.imshow(image_batch[n])
    ax.set_title(predicted_class_names[n])
    ax.axis('off')
_ = fig1.suptitle('ImageNet predictions')

Batch result shape: (32, 1001)
Batch predicted class names: ['coral reef' 'stole' 'nematode' 'nematode' 'bubble' 'nematode'
 'fire screen' 'bubble' 'brain coral' 'brain coral' 'bubble'
 'great white shark' 'chain mail' 'stole' 'bubble' 'brain coral' 'bubble'
 'nematode' 'bubble' 'screw' 'chain mail' 'bubble' 'jellyfish'
 'coral reef' 'bubble' 'starfish' 'brain coral' 'fire screen' 'bubble'
 'paper towel' 'bubble' 'nematode']


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
# Prepare transfer learning
## Download headless (without the top classification layer) model
feature_extractor_url = 'https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2' #@param {type:"string"}
feature_extractor_layer = hub.KerasLayer(feature_extractor_url,
                                         input_shape=IMAGE_SHAPE+(3, ))
feature_batch = feature_extractor_layer(image_batch)
print(f'Feature vector shape: {feature_batch.shape}')

## Frozen feature extraction layer
feature_extractor_layer.trainable = False # for transfer learning classifier

## Make a model for classification
model = tf.keras.Sequential([
    feature_extractor_layer,
    layers.Dense(train_data.num_classes, activation='softmax')
])

Feature vector shape: (32, 1280)


In [6]:
## Check the model and prediction result
model.summary()

predictions = model(image_batch)
print(f'Prediction shape: {predictions.shape}')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 3)                 3843      
Total params: 2,261,827
Trainable params: 3,843
Non-trainable params: 2,257,984
_________________________________________________________________
Prediction shape: (32, 3)


In [7]:
# Train build
## Compile model for train
base_learning_rate = 0.001 # default
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
    loss='categorical_crossentropy',
    metrics=['acc'])

## Log class
### https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/Callback
class CollectBatchStats(tf.keras.callbacks.Callback):
    def __init__(self):
        self.batch_losses = []
        self.batch_val_losses = []
        self.batch_acc = []
        self.batch_val_acc = []

#     def on_train_batch_end(self, batch, logs=None):
#         self.batch_losses.append(logs['loss'])
#         self.batch_acc.append(logs['acc'])
#         self.model.reset_metrics()
    
#     def on_test_batch_end(self, batch, logs=None):
#         self.batch_val_losses.append(logs['loss'])
#         self.batch_val_acc.append(logs['acc'])
#         self.model.reset_metrics()
    
    def on_epoch_end(self, epoch, logs=None):
        self.batch_losses.append(logs['loss'])
        self.batch_acc.append(logs['acc'])
        self.batch_val_losses.append(logs['val_loss'])
        self.batch_val_acc.append(logs['val_acc'])
        self.model.reset_metrics()


steps_per_epoch = np.ceil(train_data.samples/train_data.batch_size) # train all dataset per epoch
initial_epoch = 20
batch_stats_callback = CollectBatchStats()

history = model.fit_generator(train_data,  
                              epochs=initial_epoch,
                              steps_per_epoch=steps_per_epoch,
                              validation_data=validation_data,
                              callbacks=[batch_stats_callback])

Instructions for updating:
Please use Model.fit, which supports generators.


Instructions for updating:
Please use Model.fit, which supports generators.


  ...
    to  
  ['...']


  ...
    to  
  ['...']


  ...
    to  
  ['...']


  ...
    to  
  ['...']


Train for 27.0 steps, validate for 3 steps
Epoch 1/20
27/27 [==============================] - 30s 1s/step - loss: 1.0058 - acc: 0.5214 - val_loss: 0.7009 - val_acc: 0.6957
Epoch 2/20
27/27 [==============================] - 30s 1s/step - loss: 0.5065 - acc: 0.8242 - val_loss: 0.4808 - val_acc: 0.8043
Epoch 3/20
27/27 [==============================] - 27s 989ms/step - loss: 0.3739 - acc: 0.8800 - val_loss: 0.3708 - val_acc: 0.8913
Epoch 4/20
27/27 [==============================] - 28s 1s/step - loss: 0.2971 - acc: 0.9264 - val_loss: 0.3448 - val_acc: 0.8804
Epoch 5/20
27/27 [==============================] - 28s 1s/step - loss: 0.2566 - acc: 0.9382 - val_loss: 0.3119 - val_acc: 0.9022
Epoch 6/20
27/27 [==============================] - 28s 1s/step - loss: 0.2211 - acc: 0.9466 - val_loss: 0.3115 - val_acc: 0.9022
Epoch 7/20
27/27 [==============================] - 29s 1s/step - loss: 0.1993 - acc: 0.9525 - val_loss: 0.3130 - val_acc: 0.8913
Epoch 8/20
27/27 [==========================

In [8]:
# Draw learning curves chart
acc = batch_stats_callback.batch_acc
val_acc = batch_stats_callback.batch_val_acc
loss = batch_stats_callback.batch_losses
val_loss = batch_stats_callback.batch_val_losses

fig2 = plt.figure(figsize=(8, 8))
ax1 = fig2.add_subplot(2, 1, 1)
ax1.plot(acc, label='Training Accuracy')
ax1.plot(val_acc, label='Validation Accuracy')
ax1.legend(loc='lower right')
ax1.set_ylabel('Accuracy')
ax1.set_ylim([min(plt.ylim()),1])
ax1.set_title('Training and Validation Accuracy')

ax2 = fig2.add_subplot(2, 1, 2)
ax2.plot(loss, label='Training Loss')
ax2.plot(val_loss, label='Validation Loss')
ax2.legend(loc='upper right')
ax2.set_ylabel('Cross Entropy')
ax2.set_ylim([0,1.0])
ax2.set_title('Training and Validation Loss')
ax2.set_xlabel('epoch')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'epoch')

In [9]:
# Plot results
class_names = sorted(validation_data.class_indices.items(), key=lambda pair:pair[1])
class_names = np.array([key.title() for key, value in class_names])
print(f'Classes: {class_names}')

## get result labels
predicted_batch = model.predict(image_batch)
predicted_id = np.argmax(predicted_batch, axis=-1)
predicted_label_batch = class_names[predicted_id]

label_id = np.argmax(label_batch, axis=-1)

## plot
fig3 = plt.figure(figsize=(10,9))
fig3.subplots_adjust(hspace=0.5)
for n in range(30):
    ax = fig3.add_subplot(6, 5, n+1)
    ax.imshow(image_batch[n])
    color = 'green' if predicted_id[n] == label_id[n] else 'red'
    ax.set_title(predicted_label_batch[n].title(), color=color)
    ax.axis('off')
_ = fig3.suptitle('Model predictions (green: correct, red: incorrect)')

Classes: ['Hek293' 'Hela' 'Pc12']


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
# Export model
import time
t = time.time()

export_path = f'./saved_models/{int(t)}'
model.save(export_path, save_format='tf')

print(f'Export the model to {export_path}')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: ./saved_models/1580975332/assets


INFO:tensorflow:Assets written to: ./saved_models/1580975332/assets


Export the model to ./saved_models/1580975332


In [11]:
# Check the exported model
reloaded = tf.keras.models.load_model(export_path)

result_batch = model.predict(image_batch)
reloaded_result_batch = reloaded.predict(image_batch)

print(f'Comparison between own model and exported model {abs(reloaded_result_batch - result_batch).max()}')

Comparison between own model and exported model 0.0


In [12]:
## Unfrozen feature extraction layer
feature_extractor_layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate/10),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['acc'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 3)                 3843      
Total params: 2,261,827
Trainable params: 2,227,715
Non-trainable params: 34,112
_________________________________________________________________


In [13]:
finetune_epoch = 20

history_fine = model.fit_generator(train_data,
                                   epochs=initial_epoch+finetune_epoch,
                                   initial_epoch=initial_epoch, # == history.epoch[-1]+1
                                   steps_per_epoch=steps_per_epoch,
                                   validation_data=validation_data,
                                   callbacks = [batch_stats_callback])

  ...
    to  
  ['...']


  ...
    to  
  ['...']


  ...
    to  
  ['...']


  ...
    to  
  ['...']


Train for 27.0 steps, validate for 3 steps
Epoch 21/40
27/27 [==============================] - 67s 2s/step - loss: 0.8950 - acc: 0.7720 - val_loss: 0.4584 - val_acc: 0.8370
Epoch 22/40
27/27 [==============================] - 64s 2s/step - loss: 0.1315 - acc: 0.9905 - val_loss: 0.4150 - val_acc: 0.8587
Epoch 23/40
27/27 [==============================] - 71s 3s/step - loss: 0.0910 - acc: 1.0000 - val_loss: 0.4012 - val_acc: 0.8913
Epoch 24/40
27/27 [==============================] - 61s 2s/step - loss: 0.0868 - acc: 1.0000 - val_loss: 0.3649 - val_acc: 0.9022
Epoch 25/40
27/27 [==============================] - 62s 2s/step - loss: 0.0857 - acc: 1.0000 - val_loss: 0.3376 - val_acc: 0.9022
Epoch 26/40
27/27 [==============================] - 63s 2s/step - loss: 0.0852 - acc: 1.0000 - val_loss: 0.3084 - val_acc: 0.9022
Epoch 27/40
27/27 [==============================] - 65s 2s/step - loss: 0.0848 - acc: 1.0000 - val_loss: 0.2866 - val_acc: 0.8913
Epoch 28/40
27/27 [=====================

In [16]:
# Draw learning curves chart
fine_acc = batch_stats_callback.batch_acc
fine_val_acc = batch_stats_callback.batch_val_acc
fine_loss = batch_stats_callback.batch_losses
fine_val_loss = batch_stats_callback.batch_val_losses

fig4 = plt.figure(figsize=(8, 8))
ax1 = fig4.add_subplot(2, 1, 1)
ax1.plot(acc, label='Training Accuracy')
ax1.plot(val_acc, label='Validation Accuracy')
ax1.set_ylabel('Accuracy')
ax1.set_ylim([min(plt.ylim()),1])
ax1.plot([initial_epoch,initial_epoch],
         ax1.get_ylim(), label='Start Fine Tuning')
ax1.legend(loc='lower right')
ax1.set_title('Training and Validation Accuracy')

ax2 = fig4.add_subplot(2, 1, 2)
ax2.plot(loss, label='Training Loss')
ax2.plot(val_loss, label='Validation Loss')
ax2.set_ylabel('Cross Entropy')
ax2.set_ylim([0,1.0])
ax2.plot([initial_epoch,initial_epoch],
         ax2.get_ylim(), label='Start Fine Tuning')
ax2.legend(loc='upper right')
ax2.set_title('Training and Validation Loss')
ax2.set_xlabel('epoch')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'epoch')

In [17]:
# Plot results
class_names = sorted(validation_data.class_indices.items(), key=lambda pair:pair[1])
class_names = np.array([key.title() for key, value in class_names])
print(f'Classes: {class_names}')

## get result labels
predicted_batch = model.predict(image_batch)
predicted_id = np.argmax(predicted_batch, axis=-1)
predicted_label_batch = class_names[predicted_id]

label_id = np.argmax(label_batch, axis=-1)

## plot
fig5 = plt.figure(figsize=(10,9))
fig5.subplots_adjust(hspace=0.5)
for n in range(30):
    ax = fig5.add_subplot(6, 5, n+1)
    ax.imshow(image_batch[n])
    color = 'green' if predicted_id[n] == label_id[n] else 'red'
    ax.set_title(predicted_label_batch[n].title(), color=color)
    ax.axis('off')
_ = fig5.suptitle('Model predictions (green: correct, red: incorrect)')

Classes: ['Hek293' 'Hela' 'Pc12']


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
# Export model too
t = time.time()

export_path = f'./saved_models/{int(t)}'
model.save(export_path, save_format='tf')

print(f'Export the model to {export_path}')

INFO:tensorflow:Assets written to: ./saved_models/1580976720/assets


INFO:tensorflow:Assets written to: ./saved_models/1580976720/assets


Export the model to ./saved_models/1580976720


In [19]:
# Check the exported model too
reloaded = tf.keras.models.load_model(export_path)

result_batch = model.predict(image_batch)
reloaded_result_batch = reloaded.predict(image_batch)

print(f'Comparison between own model and exported model {abs(reloaded_result_batch - result_batch).max()}')

Comparison between own model and exported model 0.0
